# Deutsch–Jozsa Algorithm Tutorial

The **Deutsch–Jozsa algorithm** is one of the most famous algorithms in quantum computing. The problem it solves has little practical value, but the algorithm itself is one of the earliest examples of a quantum algorithm that is exponentially faster than any possible deterministic algorithm for the same problem. It is also relatively simple to explain and illustrates several very important concepts (such as quantum oracles). As such, Deutsch–Jozsa algorithm is part of almost every introductory course on quantum computing.

This tutorial will:
* introduce you to the problem solved by the Deutsch–Jozsa algorithm and walk you through the classical solution to it,
* give you a brief introduction to quantum oracles,
* describe the idea behind the Deutsch–Jozsa algorithm and walk you through the math for it,
* teach you how to implement the algorithm in the Q# programming language,
* and finally help you to run your implementation of the algorithm on several quantum oracles to see for yourself how the algorithm works!

Let's go!

To begin, first prepare this notebook for execution (if you skip the first step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells; if you skip the second step, you'll get "Invalid test name" error):

In [ ]:
%package Microsoft.Quantum.Katas::0.7.1905.3109

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


In [ ]:
%workspace reload

# Part I. Problem Statement and Classical Algorithm

## The problem

You are given a classical function $f(x): \{0, 1\}^N \to \{0, 1\}$. You are guaranteed that the function $f$ is
* either *constant* (has the same value for all inputs) 
* or *balanced* (has value 0 for half of the inputs and 1 for the other half of the inputs). 

The task is to figure out whether the function is constant or balanced.

## Examples

* $f(x) \equiv 0$ or $f(x) \equiv 1$ are examples of constant functions (and they are actually the only constant functions in existence).
* $f(x) = x \text{ mod } 2$ or $f(x) = 1 \text{ if the binary notation of }x \text{ has odd number of 1s and 0 otherwise}$ are examples of balanced functions.  
  Indeed, for both these functions you can check that for every possible input $x$ for which $f(x) = 0$ there exists an input $x^\prime$ (equal to $x$ with the least significant bit flipped) such that $f(x^\prime) = 1$, and vice versa, which means that the function is balanced.  
  There exist more complicated examples of balanced functions, but we will not need to consider them for this tutorial.

## Classical algorithm

If we solve this problem classically, how many calls to the given function will we need? 

The first call will give us no information - regardless of whether it returns 0 or 1, the function could still be constant or balanced.
In the best case scenario the second call will return a different value and we'll be able to conclude that the function is balanced in just <span style="color:red">$2$</span> calls. 
However, if we get the same value for the first two calls, we'll have to keep querying the function until either we get a different value or until we do <span style="color:red">$2^{N-1}+1$</span> queries that will return the same value - in this case we'll know for certain that the function will be constant.

## Exercise 1: Implement the classical algorithm!

Q# is a domain-specific language, so it is not designed to handle arbitrary classical computations. However, this classical algorithm is so simple that you can easily implement it in Q#. Try it!

**Inputs:** 
1. The number of bits in the input $N$ ($1 \le N \le 5$).
2. The "black box" function that evaluates $f(x)$ on any given input $x \in [0, 2^N-1]$.  
  You are guaraneed that the function implemented by the black box is either constant or balanced.

**Goal:** Return `true` if the function is constant, or `false` if it is balanced.

In [ ]:
%kata E1_ClassicalSolution_Test 

operation IsFunctionConstant_Classical (N : Int, f : (Int -> Int)) : Bool {
    // ...
}

# Part II. Quantum Oracles

## Definition
A quantum oracle is a "black box" operation which is used as input to another algorithm. This operation is implemented in a way which allows to perform calculations not only on individual inputs, but also on superpositions of inputs. 

> This is *not* the same as being able to evaluate the function on all inputs at once, since you will not be able to extract the evaluation results!

Oracles are often defined using a classical function, in the case of Deutsch-Jozsa algorithm the function $f : \{0, 1\}^N \to \{0, 1\}$ takes an $N$-bit binary input and produces an 1-bit binary output.

The oracle has to act on quantum states instead of classical values. 
To enable this, integer input $x$ is represented in binary $x = (x_{0}, x_{1}, \dots, x_{N-1})$, 
and encoded into an $N$-qubit register: $|\vec{x} \rangle = |x_{0} \rangle \otimes |x_{1} \rangle \otimes \cdots \otimes |x_{N-1} \rangle$.

The type of oracles used in this tutorial are called *phase oracles*. A phase oracle $U_f$ encodes the value of the classical function $f$ it implements in the phase of the qubit state as follows:

$$U_f |\vec{x} \rangle = (-1)^{f(x)} |\vec{x} \rangle$$

In our case $f$ can take only two values, 0 or 1, which result in no phase change or adding a $-1$ phase, respectively.

The effect of such an oracle on any single basis state is not particularly interesting: it just adds a global phase which is not something you can observe. However, if you apply this oracle to a superposition of basis states, its effect becomes noticeable. Remember that quantum operations are linear: if you define the effect of an operation on the basis states, you'll be able to deduce its effect on superposition states (which are just linear combinations of the basis states) using its linearity. 

## Example: Deutsch algorithm

Consider, for example, the case of $N = 1$: there are two possible inputs to the function, $|0\rangle$ and $|1\rangle$, and we can apply the oracle to their superposition:

$$U_f \left( \frac{1}{\sqrt2} \big( |0\rangle + |1\rangle \big) \right) 
= \frac{1}{\sqrt2} \big( U_f |0\rangle + U_f |1\rangle \big) 
= \frac{1}{\sqrt2} \big( (-1)^{f(0)} |0\rangle + (-1)^{f(1)} |1\rangle \big)$$.

* If $f(0) = f(1)$, the phases of the two basis states are the same, and the resulting state is $|+\rangle = \frac{1}{\sqrt2} \big( |0\rangle + |1\rangle \big)$ (up to a global phase). 
* If $f(0) \neq f(1)$, the phases of the two basis states differ by a factor of $-1$, and the resulting state is $|-\rangle = \frac{1}{\sqrt2} \big( |0\rangle + |1\rangle \big)$ (up to a global phase). 
* The states $|+\rangle$ and $|-\rangle$ are orthogonal and thus can be distinguished using measurements. This means that one oracle call does not let you calculate both $f(0)$ and $f(1)$, but it allows you to figure out whether $f(0) = f(1)$.

This is a special case of the Deutsch-Jozsa algorith, called Deutsch algorithm. 

## Implementing oracles in Q&#x23;

Now that we've discussed the mathematical definition of the oracles, let's take a look at how to implement oracles for some classical functions in Q#. We'll consider the same 4 functions we used as an example in the first section.

### $f(x) \equiv 0$

This is the easiest function to implement: if $f(x) \equiv 0$, $U_f |x\rangle \equiv (-1)^0 |x\rangle = |x\rangle$. This means that $U_f$ is an identity - a transformation which does absolutely nothing! This is very easy to express in Q#:

In [ ]:
operation PhaseOracle_Zero (x : Qubit[]) : Unit {
    // Do nothing...
}

### $f(x) \equiv 1$

The second constant function is slightly trickier: if $f(x) \equiv 1$, $U_f |x\rangle \equiv (-1)^1 |x\rangle = - |x\rangle$. Now $U_f$ is a negative identity, i.e., a transformation which applies a global phase of $-1$ to the state. A lot of algorithms just ignore the global phase accumulated in them, since it is not observable. However, if we want to be really meticulous, we can use Q# library operation [Microsoft.Quantum.Intrinsic.R](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.r) which performs a given rotation around the given axis. When called with `PauliI` axis, this operation applies a global phase; since it doesn't take the input into account, it can be applied to any qubit, for example, the first qubit of the input.

In [ ]:
// Open namespace where the library function PI() is defined
open Microsoft.Quantum.Math;

operation PhaseOracle_One (x : Qubit[]) : Unit {
    // Apply a global phase of -1
    R(PauliI, 2.0 * PI(), x[0]);
}

### $f(x) = x \text{ mod } 2$

In this oracle we will finally need to use the input! The binary representation of $x$ is $x = (x_{0}, x_{1}, \dots, x_{N-1})$, with the least significant bit encoded in the last bit (stored in the last qubit of the input array): $f(x) = x_{N-1}$. Let's use this in the oracle effect expression:

$$U_f |x\rangle = (-1)^{f(x)} |x\rangle = |x_{0} \rangle \otimes \cdots \otimes |x_{N-2} \rangle \otimes (-1)^{x_{N-1}} |x_{N-1}\rangle$$

This means that we only need to use the last qubit in the implementation: do nothing if it is $|0\rangle$ and apply a phase of $-1$ if it is $|1\rangle$. This is exactly the effect of the [Z gate](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.z); as a reminder, 

$$Z = \begin{bmatrix} 1 & 0 \\ 0 & -1\end{bmatrix}: Z |0\rangle = |0\rangle, Z |1\rangle = -|1\rangle$$

Finally, the expression for the oracle is:

$$U_f = \mathbb{1} \otimes \cdots \otimes \mathbb{1} \otimes Z$$

In [ ]:
operation PhaseOracle_Xmod2 (x : Qubit[]) : Unit {
    // Length(x) gives you the length of the array.
    // Array elements are indexed 0 through Length(x)-1, inclusive.
    Z(x[Length(x) - 1]);
}

### $f(x) = 1 \text{ if x has odd number of 1s, and 0 otherwise }$

In this oracle the answer depends on all bits of the input. We can write $f(x)$ as follows:

$$f(x) = \bigoplus_{k=0}^{N-1} x_k$$ 

$$U_f |x\rangle = (-1)^{f(x)} |x\rangle = \bigotimes_{k=0}^{N-1} (-1)^{x_k} |x_{k}\rangle$$

As we've seen in the previous oracle, this can be achieved by applying a Z gate to each qubit; you can use library function [ApplyToEach](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.canon.applytoeach) to apply a gate to each qubit of the array.

In [ ]:
operation PhaseOracle_OddNumberOfOnes (x : Qubit[]) : Unit {
    ApplyToEach(Z, x);
}

# Part III. Quantum Algorithm

Now that we have figured out how the oracles work and looked at Deutsch algorithm, we can get back to solving the big problem.


<span style="color:red">TODO:</span> theoretical description assuming that the oracle is implemented as phase oracle


## Exercise 2. Implement the quantum algorithm!

**Inputs:** 
1. The number of bits in the input $N$ ($1 \le N \le 5$).
2. The "black box" oracle the implements $f(x)$.  
  You are guaraneed that the function implemented by the oracle is either constant or balanced.

**Goal:** Return `true` if the function is constant, or `false` if it is balanced.

In [ ]:
%kata E2_DJAlgorithm_Test

operation DeutschJozsaAlgorithm (N : Int, oracle : (Qubit[] => Unit)) : Bool {
    // Create a boolean variable for storing the return value.
    // You'll need to update it later, so it has to be declared as mutable.
    mutable isConstant = ...;

    // Allocate an array of N qubits for the input register x.
    using (x = Qubit[...]) {
        // Newly allocated qubits start in the |0⟩ state.
        // The first step is to prepare the qubits in the required state before calling the oracle.
        // A qubit can be transformed from the |0⟩ state to the |+⟩ state by applying a Hadamard gate H.
        // ...

        // Apply the oracle to the input register.
        // The syntax is the same as for applying any function or operation.
        // ...

        // Apply a Hadamard gate to each qubit of the input register again.
        // ...

        // Measure each qubit of the input register in the computational basis using the M operation.
        // You can use a for loop to iterate over the range of indexes 0..N-1.
        // Note that you can't return the answer in the middle of a loop,
        // you have to update the variable isConstant using the "set" keyword.
        // ...

        // Before releasing the qubits make sure they are all in the |0⟩ state
        // (otherwise you'll get a ReleasedQubitsAreNotInZeroState exception).
        // You can use the library operation Reset which measures a qubit and applies a correction if necessary.
        // The library operation ResetAll does the same for a register of qubits.
        // ...
    }
    
    // Return the value of the boolean variable.
    return ...;
}

# Part IV. Running the Algorithm

You have implemented the quantum version of the algorithm - congratulations! The last step is to combine everything you've seen so far - run your code to check whether the oracles you've seen in part II implement constant or balanced functions.

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_DeutschJozsaAlgorithm` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_DeutschJozsaAlgorithm`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task. Don't forget to execute Q# code cells that define oracles in part II!

<span style="color:red">TODO:</span> a special treat: several library oracles for which we need to figure out what kind of a function they implement!

In [ ]:
open Microsoft.Quantum.Diagnostics;

operation Run_DeutschJozsaAlgorithm () : String {
    // You can use Fact function to check that the return value of DeutschJozsaAlgorithm operation matches the expected value
    Fact(DeutschJozsaAlgorithm(4, PhaseOracle_Zero) == true, "f(x) = 0 not identified as constant");
    
    // Run the algorithm for the rest of the oracles
    // ...
    
    // If all tests pass, report success!
    return "Success!";
}

In [ ]:
%simulate Run_DeutschJozsaAlgorithm